In [ ]:
#default_exp datablock

# DataBlock

> process timeseries datasets with DataBloc

In [ ]:
#using the NumpyDataLoader in tsai is significantly faster than using the fastaiv2 DataLoader, 
#maybe build a custom dataloader to optimize for odds data
import tsai
from tsai.all import *

In [ ]:
print('tsai       :', tsai.__version__)
print('fastai2    :', fastai2.__version__)
print('fastcore   :', fastcore.__version__)
print('torch      :', torch.__version__)
print('scipy      :', sp.__version__)
print('numpy      :', np.__version__)
print('pandas     :', pd.__version__)
print(f'Total RAM  : {bytes2GB(psutil.virtual_memory().total):5.2f} GB')
print(f'Used RAM   : {bytes2GB(psutil.virtual_memory().used):5.2f} GB')
print('n_cpus     :', cpus)
iscuda = torch.cuda.is_available()
if iscuda: print('device     : {} ({})'.format(device, torch.cuda.get_device_name(0)))
else: print('device     :', device)

tsai       : 0.1.0
fastai2    : 0.0.18
fastcore   : 0.1.18
torch      : 1.3.1
scipy      : 1.4.1
numpy      : 1.18.1
pandas     : 0.25.3
Total RAM  : 15.56 GB
Used RAM   :  7.51 GB
n_cpus     : 4
device     : cuda (GeForce GTX 950)


In [ ]:
#export
from torchtools.data import *
from torchtools.datasets import *
from torchtools.augmentations import *

from torchtools.datablock import *

In [ ]:
from torchtools.models import *
from torchtools.core import *

In [ ]:
import pandas as pd
import numpy as np
from functools import partial

In [ ]:
_verbose=True

### Get Data

In [ ]:
df_main = pd.read_csv('./data/custom/bi_sample_anon.csv', nrows=200000)

In [ ]:
## simple config
x_cols = [[f'x{i}_{j}' for j in range(10)] for i in range(6)]
dep = 'y0'
n_train = 8000

items is a list of (x,y) tuples  
methods called when passing items to DataBlock:
- DataBlock.datasets(items, ....)
- Dataset(items, ...)
- TfmdList(items, ...)
- L(items, ...)
- CollBase.__init__(items, ...)



### TSAI 

In [ ]:
items_0 = df_to_items(df_main, x_cols, dep, n_train)[0]

(200000, 6, 10)


In [ ]:
#export
def _items_to_arrays(items):
#     return np.stack([x[0] for x in items]), np.stack([x[1] for x in items])
    return tuple(np.stack([x[i] for x in items]) for i in range(len(items[0])))

In [ ]:
X,y = _items_to_arrays(items_0)

In [ ]:
X.shape, y.shape

((200000, 6, 10), (200000,))

In [ ]:
##use FixedSplitter
splits = [L(range(160000)), L(range(160000,185000))]
splits = FixedSplitter()(df_main)
# splits = [L(range(1600)), L(range(1600,1850))]

tfms  = [None, None]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)

In [ ]:
splits

((#10000) [0,1,2,3,4,5,6,7,8,9...],
 (#5000) [10000,10001,10002,10003,10004,10005,10006,10007,10008,10009...])

In [ ]:
# this is still a hack to transform the tsai tensor types to torchtool tensor types
# not needed when using adapted TSDataloader
@Transform
def ToTT(x:tsai.data.core.TSTensor):
    print('ToTT')
    return cast(x, TSTensor)

ToTT.order = 150
ToTT.split_idx = None

In [ ]:
#int
@Transform
def ToTTInt(x:tsai.data.core.TSTensor):
    print('ToTT')
    return cast(x, TSIntTensor)

ToTT.order = 150
ToTT.split_idx = 0

In [ ]:
# dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0, 
#                                after_batch=[TSNormalize(by_var=False)
                               batch_tfms=[TSNormalize(by_sample=False, by_var=True), ToTT])

In [ ]:
dls[0].one_batch()

ToTT


(TSTensor([[[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  0.2195,  0.2195],
          [ 0.1429,  0.1429,  0.5714,  ...,  0.1429,  0.2857,  0.2857],
          [ 1.0000,  0.5404,  1.0000,  ...,  0.2500,  0.0494,  0.4855],
          [ 1.0000, -0.5610,  1.0000,  ..., -0.7441, -0.6108,  0.2195],
          [ 0.5714, -0.1429,  0.1429,  ..., -0.1429,  0.0000, -0.2857]],
 
         [[-0.1497,  1.0000,  0.2882,  ...,  1.0000, -0.1770,  0.3746],
          [-0.6288,  1.0000, -0.5457,  ...,  1.0000,  0.2195,  0.2195],
          [ 0.0000,  0.2857, -0.4286,  ...,  0.1429,  0.0000,  0.0000],
          [ 1.0000,  0.5041,  1.0000,  ...,  1.0000,  0.3399, -0.0426],
          [-0.6477, -0.9116,  1.0000,  ...,  0.2195,  0.2195,  0.2195],
          [ 0.1429, -0.1429,  0.2857,  ...,  0.1429,  0.0000,  0.0000]],
 
         [[-0.2908,  1.0000,  1.0000,  ...,  0.5820,  0.5820,  0.5820],
          [ 0.2195,  0.2195,  0.2195,  ...,  0.2138,  0.

In [ ]:
# #export
# _batch_tfms = ('after_item','before_batch','after_batch')

# class NumpyDataLoaders(DataLoaders):
#     _xblock = NumpyTensorBlock
#     _dl_type = NumpyDataLoader 
#     def __init__(self, *loaders, path='.', device=default_device()):
#         self.loaders,self.path = list(loaders),Path(path)
#         self.device = device
        
#     @classmethod
#     @delegates(DataLoaders.from_dblock)
#     def from_numpy(cls, X, y=None, splitter=None, valid_pct=0.2, seed=0, item_tfms=None, batch_tfms=None, **kwargs):
#         "Create timeseries dataloaders from arrays (X and y, unless unlabeled)"
#         if splitter is None: splitter = RandomSplitter(valid_pct=valid_pct, seed=seed)
#         getters = [ItemGetter(0), ItemGetter(1)] if y is not None else [ItemGetter(0)]
#         dblock = DataBlock(blocks=(cls._xblock, CategoryBlock),
#                            getters=getters,
#                            splitter=splitter,
#                            item_tfms=item_tfms,
#                            batch_tfms=batch_tfms)

#         source = itemify(X) if y is None else itemify(X,y)
#         return cls.from_dblock(dblock, source, **kwargs)

In [ ]:
norm = TSNormalize(by_sample=False, by_var=True)
tszero = TimestepZero(verbose=True)

In [ ]:
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0, 
                              after_batch=Pipeline(noop),
                            batch_tfms=[norm, ToTT, tszero])

In [ ]:
dls[1].one_batch()

(TSTensor(samples:512, vars:6, len:10),
 tensor([ -41.6667,  100.0000, -131.5789,  100.0000,  100.0000,  -25.6410,
          100.0000,  -25.3165,  100.0000,  -32.7869,  100.0000, -256.4102,
          -35.7143,  100.0000,  100.0000,  -21.2766,  100.0000,   -6.7797,
         -153.8462,  100.0000,  100.0000,  -34.6021,  100.0000,  -25.7732,
          100.0000,  -18.1159,  100.0000,  -64.9351, -163.9344,  100.0000,
          100.0000,  -16.5563,  -96.1538,  100.0000,  -94.3396,  -29.6736,
          -72.4638,  100.0000,  100.0000,  -77.5194, -131.5789,  100.0000,
          -64.1026,  100.0000,  100.0000,  -36.1011,  -63.6943,  -49.7512,
          100.0000,  -41.8410,  -34.6021,  100.0000,  -15.1515,  100.0000,
          100.0000,  -21.6920,  -76.3359,  -35.2113,  100.0000,  -41.1523,
          100.0000,  -32.7869,  100.0000,  -28.0112, -322.5807,   -9.0909,
          -56.1798,  -53.4759,  -66.6667,  -50.2513,  -48.3092,  -54.3478,
          100.0000,  -27.0270,  100.0000,  -20.9644,  100.00

In [ ]:
model = InceptionTimeSgm(6,1)
loss_fn = get_loss_fn('leaky_loss', alpha=0.5)
learn_small = Learner(model=model, loss_func=loss_fn, dls=dls, metrics=unweighted_profit)

In [ ]:
learn_small.fit_one_cycle(5)

epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.427778,-0.211007,-0.537000,00:02
1,-1.045388,-0.839335,0.621834,00:02
2,-1.446260,-0.918100,0.475411,00:02
3,-2.104608,-1.053528,0.481733,00:02
4,-3.343295,-0.942993,0.421245,00:02


ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
timestepzero
ToTT
times

In [ ]:
preds = learn_small.get_preds(1)

In [ ]:
unweighted_profit(*preds)

tensor(0.4212)

#### Modelling with RandAugment

In [ ]:
tfms = all_erasing_augs(magnitude=0.4) + all_noise_augs(magnitude=0.4) + all_zoom_augs(magnitude=0.4)

In [ ]:
splits = [L(range(160000)), L(range(160000,185000))]
# splits = [L(range(1600)), L(range(1600,1850))]

# tfms  = [None, None]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)

# dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 512], num_workers=0)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[256, 256], num_workers=0, 
#                                after_batch=[TSNormalize(by_var=False)
                               batch_tfms=[TSNormalize(by_sample=False, by_var=True), ToTT, 
                                           RandAugment(N=3, magnitude=0.4, tfms=tfms)
                                           ])

tfms [Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Cutout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , TimestepZero: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Crop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , RandomCrop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Maskout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , TimeNormal: (TSTensor,object) -> encodes , Zoomin: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , Zoomout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , RandZoom: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes , RandTimesteps:

In [ ]:
xb, yb = dls[0].one_batch()

ToTT


In [ ]:
dls.after_batch.tfms[3].magnitude

0.4

In [ ]:
# dls.show_batch()

In [ ]:
set_seed(1234)

In [ ]:
## for tsai
model = InceptionTimeSgm(6,1)
learn = Learner(dls, model, loss_fn, metrics=[unweighted_profit, partial(unweighted_profit, threshold=0.5)])

In [ ]:
learn.fit_one_cycle(2, lr_max=1e-5, wd=0.03)

epoch,train_loss,valid_loss,unweighted_profit,unweighted_profit,time
0,-0.088209,-0.120086,0.003209,0.000000,00:40
1,-0.284319,-0.147903,0.032609,0.000000,00:36


ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT
ToTT


In [ ]:
preds = learn.get_preds()

In [ ]:
(preds[0]>-0.5).sum()

tensor(23892)

In [ ]:
unweighted_profit(*preds, threshold=0)

tensor(0.0326)

### Discrete

oguiza TSDatasets, possible modification
- allow to directly use torchtools tensor types
- allow more than one X

- toarray, to3darray, totensor in tsai.utils useful for handling cpu, cuda, 2d, 3d cases, also for augmentations
- to3darray: a[:, None] assumes that the 2d array has batch, sequence dimensions, i.e. channel dimension missing

In [ ]:
#int
x_cont, x_dis, dep, n_train = get_discrete_config()

In [ ]:
#int
items_d, _ = df_to_items_discrete(df_main, [x_cont, x_dis], dep, n_train)

#int
X_cont, X_dis, y = _items_to_arrays(items_d)
X_cont.shape, X_dis.shape, y.shape

assert np.issubdtype(X_dis.dtype, np.integer), 'discrete array dtype should be an np.integer subtype'

(200000, 4, 10) (200000, 2, 10)


In [ ]:
a = np.random.randn(2,10)

In [ ]:
a.shape, a[None].shape, a[:, None].shape

((2, 10), (1, 2, 10), (2, 1, 10))

In [ ]:
type(dsets.items[0][0])

numpy.ndarray

NOTE: slicing into the torchtools tensory types does not preserve the type but casts them to pytorch tensors
tsai subtypes define __getitem__ like below for that reason
```
def __getitem__(self, idx):
        res = super().__getitem__(idx)
        return res.as_subclass(type(self))
```

In [ ]:
class TSDatasets(NumpyDatasets):
    "A dataset that creates tuples from X (and y) and applies `item_tfms`"
    _xtype, _xdistype, _ytype = TSTensor, TSIntTensor, None # Expected X and y output types (torch.Tensor - default - or subclass)
    def __init__(self, X=None, X_dis=None, y=None, items=None, sel_vars=None, sel_steps=None, tfms=None, tls=None, n_inp=None, dl_type=None,
                 inplace=True, **kwargs):
        self.inplace = inplace
        if tls is None:
            X = itemify(to3darray(X), tup_id=0)
            X_dis = itemify(to3darray(X_dis), tup_id=0) if X_dis is not None else X_dis
            y = itemify(y, tup_id=0) if y is not None else y
            items = tuple((X,)) if y is None else tuple((X,y))
            if X_dis is not None: items = tuple((X, X_dis, y)) if y is not None else tuple(X, X_dis,)
            self.tfms = L(ifnone(tfms,[None]*len(ifnone(tls,items))))
            
#         if X_dis is not None: self.X_dis = X_dis
       
        self.sel_vars = ifnone(sel_vars, slice(None))
        self.sel_steps = ifnone(sel_steps,slice(None))
        self.tls = L(tls if tls else [TfmdLists(item, t, **kwargs) for item,t in zip(items,self.tfms)])
        self.n_inp = (1 if len(self.tls)==1 else len(self.tls)-1) if n_inp is None else n_inp
        if len(self.tls[0]) > 0:
            _tls_types = [self._xtype, self._ytype] if len(self.tls)==2 else [self._xtype, self._xdistype, self._ytype]
#             print(_tls_types)
#             print(len(self.tls))
#             for tl,_typ in zip(self.tls, _tls_types):
#                 print (len(tl), _typ, type(tl[0]), isinstance(tl[0], torch.Tensor))
            self.types = L([ifnone(_typ, type(tl[0]) if isinstance(tl[0], torch.Tensor) else tensor) for 
                            tl,_typ in zip(self.tls, _tls_types)])
            self.ptls = L([tl if not self.inplace else tl[:] if type(tl[0]).__name__ == 'memmap' else 
                           tensor(stack(tl[:])) for tl in self.tls])

    def __getitem__(self, it):
        
#         for i,(ptl,typ) in enumerate(zip(self.ptls,self.types)):
#             print (i, typ)
        
#         return tuple([typ(ptl[it])[...,self.sel_vars, self.sel_steps] if i==0 else 
#                       typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
        ## do not enable slicing for now 
        return tuple([typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
    

    def subset(self, i): return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp, 
                                           inplace=self.inplace, tfms=self.tfms,
                                           sel_vars=self.sel_vars, sel_steps=self.sel_steps)
    @property
    def vars(self): return self[0][0].shape[-2]
    @property
    def len(self): return self[0][0].shape[-1]


In [ ]:
splits = FixedSplitter()(df_main)
list(map(len,splits))

[10000, 5000]

In [ ]:
## smaller ground dataset
x1, x2, y2 = X_cont[:20000], X_dis[:20000], y[:20000]

In [ ]:
tfms=None
dsets_d = TSDatasets(X=x1, X_dis=x2, y=y2, tfms=tfms, splits=FixedSplitter()(df_main), inplace=True)

In [ ]:
dsets_d.tfms

(#3) [None,None,None]

In [ ]:
norm = TSNormalize(by_sample=False, by_var=True)
tszero = TimestepZero(verbose=True)

In [ ]:
@Augmix
def encodes(self, x:TSIntTensor):
    return cast(self.__call__(TSTensor(x.float())).int(), type(x))

In [ ]:
augmix = Augmix(verbose=True)

tfms None


In [ ]:
augmix.tfms

[YWarp: (TSIntTensor,object) -> encodes
 (TSTensor,object) -> encodes , YNormal: (TSIntTensor,object) -> encodes
 (TSTensor,object) -> encodes , YScale: (TSIntTensor,object) -> encodes
 (TSTensor,object) -> encodes , TimeWarp: (TSIntTensor,object) -> encodes
 (TSTensor,object) -> encodes , TimeNormal: (TSTensor,object) -> encodes ]

In [ ]:
xd[0]

tensor([[ 0,  0,  0,  3,  1,  2, -1, -1,  1,  0],
        [ 2, -2,  0, -3,  3, -2, -2,  2,  1, -1]], device='cuda:0')

In [ ]:
xd

TSIntTensor([[[ 0,  0,  0,  ..., -1,  1,  0],
         [ 2, -2,  0,  ...,  2,  1, -1]],

        [[ 1,  0, -2,  ...,  0, -3,  5],
         [ 2,  0,  0,  ..., -2,  0, -3]],

        [[-1,  3, -3,  ..., -1,  1,  1],
         [-1,  1,  0,  ..., -1,  0,  1]],

        ...,

        [[ 1,  1,  2,  ...,  0,  0,  0],
         [-2, -1, -2,  ...,  1,  0,  2]],

        [[-2,  0, -3,  ...,  0,  0,  1],
         [-2,  1,  0,  ...,  0,  0,  4]],

        [[ 0,  0,  0,  ...,  0,  2,  3],
         [ 0,  0,  2,  ...,  1,  0, -3]]], device='cuda:0')

In [ ]:
augmix(xd)[0]

weights [0.39461015 0.28242359 0.32296627], m 0.2661396293905798
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
ywarp
timenormal torch.Size([256, 2, 10])
ops in step 1 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
ops in step 2 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp


tensor([[ 0,  0,  0,  2,  1,  1,  0, -1,  0,  0],
        [ 1, -1,  0, -2,  2, -1, -2,  1,  0,  0]], device='cuda:0',
       dtype=torch.int32)

In [ ]:
dls_d = TSDataLoaders.from_dsets(dsets_d.train, dsets_d.valid, bs=[256, 512], num_workers=0, 
                              after_batch=Pipeline(noop),
                            batch_tfms=[norm, ToTT, augmix])

In [ ]:
xc, xd, y = dls_d[0].one_batch()

weights [0.50549484 0.46092898 0.03357618], m 0.49902090803471816
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
ops in step 2 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
weights [0.85618547 0.053768   0.09004653], m 0.6764267086731014
ops in step 0 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ynormal
yscale
timenormal torch.Size([256, 2, 10])
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
ynormal
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 2, 10

In [ ]:
xc.shape, xd.shape, y.shape

(torch.Size([256, 4, 10]), torch.Size([256, 2, 10]), torch.Size([256]))

In [ ]:
# #export
# class InceptionTimeD(nn.Module):
#     '''
#     add a sigmoid layer to InceptionTime to get the ouput in a certain range
#     '''
    
#     def __init__(self, n_in, n_out):
#         super().__init__()
#         self.mod = nn.Sequential(InceptionTime(n_in, n_out), Sigmoid(-1., 1.))
        
#     def forward(self, xc, xd):
#         x = torch.cat([xc.float(), xd.float()], dim=-2)
#         x = x.float()
#         print(f'InceptionTimeSgm dtype {x.dtype}')
#         return self.mod(x)

In [ ]:
model = InceptionTimeD(6,1)
loss_fn = get_loss_fn('leaky_loss', alpha=0.5)

In [ ]:
learn = Learner(dls_d, model, loss_fn, metrics=[unweighted_profit, partial(unweighted_profit, threshold=0.5)])

In [ ]:

learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,unweighted_profit,unweighted_profit,time
0,0.084339,-0.503737,0.287363,0.287363,00:04
1,0.048007,-0.388225,-0.399168,0.221982,00:04
2,-0.102091,-0.125452,0.039632,0.039632,00:04
3,-0.218451,-0.419497,0.221982,0.221982,00:04
4,-0.361746,-0.419235,0.221982,0.221982,00:04


weights [0.15487448 0.32985151 0.51527401], m 0.2509213468625352
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ynormal
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 4, 10])
timewarp
weights [0.44564088 0.0129572  0.54140192], m 0.9389955749991228
ops in step 0 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
timewarp
ops in step 1 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
ynormal


weights [0.00139337 0.9154946  0.08311203], m 0.6332814953319451
ops in step 0 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timewarp
yscale
ops in step 1 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
ywarp
timenormal torch.Size([256, 4, 10])
ops in step 2 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ywarp
weights [0.19412081 0.51541526 0.29046393], m 0.7305892617863522
ops in step 0 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes 

weights [0.70201996 0.20149892 0.09648113], m 0.3577606331321787
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ywarp
ops in step 1 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
yscale
ywarp
ops in step 2 [TimeNormal: (TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
timewarp
weights [0.32557258 0.34329688 0.33113055], m 0.525389511773879
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
ywarp
timenormal torch.Size([256, 2, 10])
ops in step 1 [YWarp: (TSInt

weights [0.15303745 0.16623991 0.68072264], m 0.7866409216089238
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ynormal
timewarp
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 4, 10])
ywarp
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
weights [0.29969472 0.45977712 0.24052816], m 0.6777128321722152
ops in step 0 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
timewarp
ops in step 1 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TST

...

weights [0.09847321 0.89510694 0.00641986], m 0.07488777352739777
ops in step 0 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
weights [0.22113665 0.51511252 0.26375083], m 0.7669278339341468
ops in step 0 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
yscale
ops in step 2 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
InceptionTimeSgm dtype torch.float32
weights [0.09753758 0.41064537 0.49181705], m 0.8835288176241481
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> en

weights [0.571009   0.34159544 0.08739556], m 0.6437062249865199
ops in step 0 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ynormal
ywarp
timenormal torch.Size([256, 4, 10])
ops in step 2 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ynormal
yscale
weights [0.27741624 0.01316976 0.709414  ], m 0.6604977472477636
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes ]
timenormal torch.Size([256, 2, 10])
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]


weights [0.42730533 0.47179161 0.10090306], m 0.5703502931629647
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
timenormal torch.Size([256, 4, 10])
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 4, 10])
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timenormal torch.Size([256, 4, 10])
yscale
weights [0.50183157 0.49362334 0.00454509], m 0.33700033680947944
ops in step 0 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
ops in step 1 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,o

weights [0.74107724 0.12139491 0.13752785], m 0.0554919196350367
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ynormal
yscale
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
ywarp
timenormal torch.Size([256, 4, 10])
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
timewarp
weights [0.0039724  0.81297147 0.18305612], m 0.6082991511566817
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
timewarp
timenormal torch.Size([256, 2, 10])
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
ops in st

weights [0.6368481  0.26360939 0.0995425 ], m 0.14157664836880587
ops in step 0 [TimeNormal: (TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
ynormal
yscale
ops in step 1 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
ops in step 2 [TimeNormal: (TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 4, 10])
yscale
weights [0.15578422 0.83091898 0.0132968 ], m 0.42643678995168005
ops in step 0 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
timewarp
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([256, 2, 10])
timewarp
ops in 

In [ ]:
preds = learn.get_preds()

InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32
InceptionTimeSgm dtype torch.float32


In [ ]:
unweighted_profit(*preds)

tensor(0.2220)

In [ ]:
(preds[0]>0).sum()

tensor(302)

In [ ]:
preds[0][:40]

tensor([[-1.0000],
        [-1.0000],
        [-1.0000],
        [-1.0000],
        [-1.0000],
        [-1.0000],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [ 1.0000],
        [-1.0000],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915],
        [-0.0915]])